Keith Chamberlain
Tue June 8: The wee hours

The purpose of this repo is to document the data exploration and analyses of Lending Club data for accepted and rejected loans, provided on Kaggle by [wordsforthewise](https://www.kaggle.com/wordsforthewise). The data span from 2007 to 2018 q4. 

# Created Repo

Generated new repo "To_Loan_or_Not" (To Loan... That is the question). Additions made to the .gitignore file to include the /data folder, as well as the jupyter notebooks save files and directory. 

# Data File Exploration

    Lending_Club.zip downloaded to the data folder. Unzipped *accepted_2007_to_2018Q4.csv.gz* (392.6 Mb) and *rejected_2007_to_2018Q4.csv.gz* (255.5 Mb) gzip compressed files totalling 648.1 Mb. This is missleading, as these are compressed files. Uncompressed, as accepted.csv and rejected.csv, these files are 1.68 Gb and 1.78 Gb on this system.


In [1]:
!pwd

/Users/keithchamberlain/DSI/capstones/To_Loan_or_Not/notebooks


In [215]:
!head -n 2 ../data/accepted.csv

id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,to

In [7]:
!head -n 4 ../data/rejected.csv

Amount Requested,Application Date,Loan Title,Risk_Score,Debt-To-Income Ratio,Zip Code,State,Employment Length,Policy Code
1000.0,2007-05-26,Wedding Covered but No Honeymoon,693.0,10%,481xx,NM,4 years,0.0
1000.0,2007-05-26,Consolidating Debt,703.0,10%,010xx,MA,< 1 year,0.0
11000.0,2007-05-27,Want to consolidate my debt,715.0,10%,212xx,MD,1 year,0.0


## Columns status

There are major differences between the columns of the two, separate datasets. Most of the columns in the *accepted.csv* data are related to loan status over time, and most of those values are missing. I was hoping to compare more features between the datasets. I don't have a data dictionary, as Lending Club is no longer providing a description of the columns on their website. Right now I can see the following shared between them with Rejected on the left: 

  - Amout requested | Loand Amount (close enough perhaps?)
  - Risk_Score | fico_range_high, fico_range_low (their average?)
  - Debt-To-Income Ratio | dti
  - Zip Code | zip_code
  - State | addr_state
  - Employment Length | emp_length
  
As an alternative to relating the accepted/rejected groups, there are plenty of data in the accepted list to predict who defaulted. In any event, time to import some data in Python!

In [142]:
''' Imports '''
import numpy as np
import pandas as pd
import os
import timeit as time
import matplotlib.pyplot as plt

In [143]:
!pwd

/Users/keithchamberlain/DSI/capstones/To_Loan_or_Not/notebooks


## Dataset Descriptions

These data, as stated, differ in content for each file.

### Accepted Loans Data File

Accepted.csv is 151 columns x 260,702 rows. Of the 151 column, 113 columns import as numeric automatically. Most of these columns have something to do with the life of the loan rather than the decision to take on the loan in the first place. Most of these column values are `NaN`. These columns could be interesting regarding whether the loanee defaulted over the life of the loan, and whether that could be predicted, however, there is not enough time to assess all of these scenarios. Thus, I will focus on the shared features between the accepted and rejected loans. 

### Rejected Loans Data File

Rejected.csv is 9 columns x 2,260,702 rows. Of the 9 columns, three import as numeric, of which two vary: Amount Requested, and Risk_Score. Debt-to-Income Ratio and Zipcode imported as "Object". 

Debt-to-Income Ratio has a % sign in it. Pandas doesn't coerce that type to numeric, so I'll strip out the % in the OS. The 2 min 13 seconds is time well spent to prevent having to do this during each import with regex and the python engine instead of the C engine (slows things down to minutes instead of 15 s).


In [214]:
!pwd
t0 = time.default_timer()
!cat ../data/rejected.csv | tr -d '[\%]' > ../data/newrej.csv
t1 = time.default_timer() - t0
print(t1)

/Users/keithchamberlain/DSI/capstones/To_Loan_or_Not/notebooks
133.49025585499476


In [145]:
''' 

Grab the number of records in each file using the OS commands, then import the data into python. Record the
time taken and the number of rows for each dataset. 

'''
t0 = time.default_timer()

!wc -l ../data/accepted.csv > ../data/accepted.wc
!wc -l ../data/rejected.csv > ../data/rejected.wc
acceptedwc = ps.read_csv("../data/accepted.wc", delimiter=" ", header = None).loc[0,1]
rejectedwc = ps.read_csv("../data/rejected.wc", delimiter=" ", header = None).loc[0,1]
t1 = time.default_timer() - t0
print(t1, acceptedwc, rejectedwc)

2.9254004350004834 2260702 27648742


## Grab the Headers from the files & Select Needed Columns

In [241]:
''' Grab the number of columns in each file '''

accept_header = ps.read_csv("../data/accepted.csv", delimiter=",", nrows=0)
reject_header = ps.read_csv("../data/rejected.csv", delimiter=",", nrows=0)
print(accept_header.columns[[2, 27, 28, 24, 22, 23, 11]])
print(accept_header.columns[12:])
print(reject_header.columns)
print(reject_header.columns[[0, 3, 4, 5, 6, 7]])

Index(['loan_amnt', 'fico_range_low', 'fico_range_high', 'dti', 'zip_code',
       'addr_state', 'emp_length'],
      dtype='object')
Index(['home_ownership', 'annual_inc', 'verification_status', 'issue_d',
       'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose', 'title',
       ...
       'hardship_payoff_balance_amount', 'hardship_last_payment_amount',
       'disbursement_method', 'debt_settlement_flag',
       'debt_settlement_flag_date', 'settlement_status', 'settlement_date',
       'settlement_amount', 'settlement_percentage', 'settlement_term'],
      dtype='object', length=139)
Index(['Amount Requested', 'Application Date', 'Loan Title', 'Risk_Score',
       'Debt-To-Income Ratio', 'Zip Code', 'State', 'Employment Length',
       'Policy Code'],
      dtype='object')
Index(['Amount Requested', 'Risk_Score', 'Debt-To-Income Ratio', 'Zip Code',
       'State', 'Employment Length'],
      dtype='object')


In [228]:
print(accepted_data['zip_code'])
print(rejected_data['Zip Code'])

0    293xx
1    786xx
2    275xx
3    117xx
4    125xx
5    880xx
6    114xx
7    215xx
8    710xx
9    331xx
Name: zip_code, dtype: object
0    559xx
1    457xx
2    630xx
3    953xx
4    681xx
5    301xx
6    434xx
7    104xx
8    972xx
9    334xx
Name: Zip Code, dtype: object


## n% Random Sample

To start, we will be taking a 0.1% random sample of each of the 1.68, and 1.78 Gb files in order to work with something in memory within reasonable time frames. The row 0 (headers) needs to be ensured to be a part of that set, so the header does not get mixed up with the data and throw off the data types. 

In order to take a 0.1% random sample, I will *skip* a 0.9% random sample of rows. 

In [203]:
''' Import the files and work with a random sample '''

rndm = np.random.default_rng()
pct_rnd_sample = 0.1
    #skip = sorted(random.sample(range(n),n-s))
    #df = pandas.read_csv(filename, skiprows=skip)
t0 = time.default_timer()
accepted_skip = sorted(rndm.choice(a = acceptedwc, size = acceptedwc - int(pct_rnd_sample*acceptedwc), 
                                   replace=False))
t1 = (time.default_timer(), time.default_timer()-t0)
rejected_skip = sorted(rndm.choice(a = rejectedwc, size = rejectedwc - int(pct_rnd_sample*rejectedwc), 
                                   replace=False))
t2 = time.default_timer() - t1[0]
print(t1[1], t2)

1.7130328040002496 32.1895744229987


### Ensure Row 0 is Always Skipped

In [204]:
accepted_skip2 = [0]
rejected_skip2 = [0]
if not 0 in accepted_skip:
    accepted_skip2.extend(list(accpted_skip))
    accepted_skip = np.array(accepted_skip2)
if not 0 in rejected_skip:
    rejected_skip2.extend(list(rejected_skip))
    rejected_skip = np.array(rejected_skip2)

In [207]:
'''
Do the import!!! Using python engine due to complex regex delimiter (to get rid of % sign in one of the fields)

'''
t0 = time.default_timer()
accepted_data = pd.read_csv("../data/accepted.csv", delimiter=",", header = None, skiprows=accepted_skip, 
                            names=accept_header.columns, low_memory=False, nrows=10)
t1 = time.default_timer() - t0
print(t1)

0.9535314949971507


In [208]:
'''
Do the import!!!

'''
t0 = time.default_timer()
rejected_data = pd.read_csv("../data/newrej.csv", delimiter=",", header = None, skiprows=rejected_skip, 
                            names=reject_header.columns, low_memory=False, nrows=10)
t1 = time.default_timer() - t0
print(t1)

5.5132277099983185


In [209]:
accepted_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Columns: 151 entries, id to settlement_term
dtypes: float64(126), int64(1), object(24)
memory usage: 11.9+ KB


In [210]:
rejected_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Amount Requested      10 non-null     float64
 1   Application Date      10 non-null     object 
 2   Loan Title            10 non-null     object 
 3   Risk_Score            10 non-null     float64
 4   Debt-To-Income Ratio  10 non-null     object 
 5   Zip Code              10 non-null     object 
 6   State                 10 non-null     object 
 7   Employment Length     10 non-null     object 
 8   Policy Code           10 non-null     float64
dtypes: float64(3), object(6)
memory usage: 848.0+ bytes


In [212]:
accepted_data.describe()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,desc,dti,...,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
count,1.000000e+01,0.0,10.00000,10.00000,10.00000,10.000000,10.000000,10.000000,0.0,10.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,6.833433e+07,NaN,17515.00000,17515.00000,17515.00000,12.838000,479.321000,85600.000000,NaN,20.232000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,3.805503e+05,NaN,7947.88756,7947.88756,7947.88756,5.695251,221.227373,41069.588641,NaN,7.553098,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,6.727548e+07,NaN,1400.00000,1400.00000,1400.00000,5.320000,47.100000,59000.000000,NaN,6.790000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,6.835817e+07,NaN,16000.00000,16000.00000,16000.00000,8.490000,373.812500,64250.000000,NaN,17.777500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,6.843666e+07,NaN,20000.00000,20000.00000,20000.00000,12.880000,537.035000,70000.000000,NaN,19.835000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,6.849927e+07,NaN,22250.00000,22250.00000,22250.00000,15.540000,646.860000,82500.000000,NaN,23.065000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,6.859618e+07,NaN,27500.00000,27500.00000,27500.00000,24.240000,701.010000,195000.000000,NaN,34.950000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [159]:
rejected_data.describe()

,Amount Requested,Risk_Score,Policy Code
count,2.764873e+06,915339.000000,2.764784e+06
mean,1.313157e+04,628.346141,6.320928e-03
std,1.498862e+04,89.567166,1.122582e-01
min,0.000000e+00,0.000000,0.000000e+00
25%,5.000000e+03,591.000000,0.000000e+00
50%,1.000000e+04,637.000000,0.000000e+00
75%,2.000000e+04,675.000000,0.000000e+00
max,1.200000e+06,990.000000,2.000000e+00


/Users/keithchamberlain/DSI/capstones/To_Loan_or_Not/notebooks
